# TODOs

1. Use abstract to tell more info e.g. "tell me more", "what is the talk about?", "please tell me more details about this talk"

# Introduction

# Setup

Navigate to project's root directory:

In [1]:
%cd ..

/app


Setup project directory:

In [2]:
# folder containing all files to develop chatbot
demo_path = "demo_confbot"

In [3]:
%mkdir -p ./{demo_path}
%cd demo_confbot
!rasa init --no-prompt

/app/demo_confbot
Welcome to Rasa! 🤖

To get started quickly, an initial project will be created.
If you need some help, check out the documentation at https://rasa.com/docs/rasa.

Created project directory at '/app/demo_confbot'.
Finished creating project structure.
Training an initial model...
2019-07-22 12:32:56 INFO     rasa.model  - Data (core-config) for Core model changed.
2019-07-22 12:32:56 INFO     rasa.model  - Data (nlu-config) for NLU model changed.
Training Core model...
Processed trackers: 100%|██████████| 4/4 [00:00<00:00, 829.45it/s, # actions=14]
Processed actions: 14it [00:00, 2433.60it/s, # examples=14]
Processed trackers: 100%|████████| 94/94 [00:00<00:00, 249.14it/s, # actions=62]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 19)             0         
_________________________________________________________________
lstm (LSTM)             

62/62 [==============================] - 0s 834us/sample - loss: 1.3082 - acc: 0.4839
Epoch 63/100
62/62 [==============================] - 0s 574us/sample - loss: 1.2700 - acc: 0.5161
Epoch 64/100
62/62 [==============================] - 0s 697us/sample - loss: 1.2664 - acc: 0.4839
Epoch 65/100
62/62 [==============================] - 0s 675us/sample - loss: 1.2634 - acc: 0.5000
Epoch 66/100
62/62 [==============================] - 0s 1ms/sample - loss: 1.2676 - acc: 0.4677
Epoch 67/100
62/62 [==============================] - 0s 1ms/sample - loss: 1.2494 - acc: 0.4516
Epoch 68/100
62/62 [==============================] - 0s 2ms/sample - loss: 1.2086 - acc: 0.5161
Epoch 69/100
62/62 [==============================] - 0s 2ms/sample - loss: 1.2266 - acc: 0.5161
Epoch 70/100
62/62 [==============================] - 0s 501us/sample - loss: 1.1974 - acc: 0.5323
Epoch 71/100
62/62 [==============================] - 0s 825us/sample - loss: 1.1712 - acc: 0.5323
Epoch 72/100
62/62 [===========

# Libraries

For chatbot ML stuff:

In [4]:
import rasa.nlu
import rasa.core

In [5]:
assert rasa.nlu.__version__ == rasa.core.__version__ == '1.1.7'

For custom actions:

In [6]:
import sqlite3 as lite
import pandas as pd

# Objective

Define a happy path to guide development:

    U: Hello  
    A: Hello! I am a conference assistant ready to help you find talks to attend or anwers any questions you have.  
    U: What session would you recommend for Data Scientists today?  
    A: You can attend 'Building AI assistants that scale using machine learning and open source tools' session!  
    U: Who is the speaker and how long is the session?  
    A: The speaker is Justina. The session is 3 hours long.  
    U: Thanks!  
    A: You are very welcome!

Note:

U: User

A: Assistant (the chatbot)

# Basics

## Train Rasa's NLU component 

The goal of this step is to enable the chatbot to extract a user's intent and any relevant entities given a user input.

First, we define some training data in markdown format (click [here](https://rasa.com/docs/rasa/nlu/training-data-format/) for details of this format) and save it to the `data` folder

In [7]:
nlu_md = """
## intent: greet
- Hello
- hi
- Hey
- Good morning
- Good afternoon
- hey

## intent: goodbye
- goodbye
- bye
- bye bye
- Talk to you later
- See you later

## intent: thanks
- thanks
- thank you
- thank you so much
- that's super useful, thanks
- this is great, thank you

## intent: recommend_session
- What talks could you recommend to [data scientist](relevant_audience)?
- I am a [machine learning engineer](relevant_audience), what talks would you recommend me to attend?
- Are there any good talks you would recommend to [software engineers](relevant_audience) to go to?
- Please recommend a talk to a [data scientist](relevant_audience)?

## intent: speaker
- Who is the speaker?
- Please tell me the name of the presenter?
- Can you tell me who is the presenter?
- I would like to know the name of the speaker?
- Tell me the name of the speaker

## intent: length
- How long is the session?
- Please tell me how long is this session?
- Can you tell me how long is this session?
- How long is this presentation?

## intent: abstract
- What is the abstract?
- I would like to know more about this session, show me the abstract please
- Can I read the abstract of the talk?
- What is this talk about?
- Tell me more about this talk

## intent: inform
- to [ML engineer](relevant_audience)
- relevant to [machine learning engineers](relevant_audience:ML)
- to [SWE](relevant_audience) people

"""

%store nlu_md > ./data/nlu.md

Writing 'nlu_md' (str) to file './data/nlu.md'.


Next, we define an NLU pipeline in `config.yml`. The pipeline should contain components we want to use for tokenization, entity extraction, building features for intent classification and actually doing the intent classification.

Click [here](https://rasa.com/docs/rasa/nlu/components) for details of all built-in components in Rasa.

In [8]:
configuration = """
language: "en"

pipeline:
- name: "WhitespaceTokenizer"
- name: "CRFEntityExtractor"
- name: "CountVectorsFeaturizer"
- name: "EmbeddingIntentClassifier"
  epochs: 300

""" 

%store configuration > config.yml

Writing 'configuration' (str) to file 'config.yml'.


Now we are ready to train the NLU component:

In [9]:
!rasa train nlu

Training NLU model...
2019-07-22 12:33:39 INFO     rasa.nlu.training_data.loading  - Training data format of '/tmp/tmpx1pbbiuf/7947d7cc4cef44fdaf3d830e70225d90_nlu.md' is 'md'.
2019-07-22 12:33:39 INFO     rasa.nlu.training_data.training_data  - Training data stats: 
	- intent examples: 37 (8 distinct intents)
	- Found intents: 'goodbye', 'thanks', 'greet', 'inform', 'recommend_session', 'speaker', 'abstract', 'length'
	- entity examples: 7 (1 distinct entities)
	- found entities: 'relevant_audience'

2019-07-22 12:33:39 INFO     rasa.nlu.model  - Starting to train component WhitespaceTokenizer
2019-07-22 12:33:39 INFO     rasa.nlu.model  - Finished training component.
2019-07-22 12:33:39 INFO     rasa.nlu.model  - Starting to train component CRFEntityExtractor
2019-07-22 12:33:39 INFO     rasa.nlu.model  - Finished training component.
2019-07-22 12:33:39 INFO     rasa.nlu.model  - Starting to train component CountVectorsFeaturizer
2019-07-22 12:33:39 INFO     rasa.nlu.model  - Finishe

We can get a feel of how good the NLU component is by manually executing the following command in a shell:

    rasa shell nlu
    
and passing it sample user inputs.

For example, here is the result of the bot's evaluation of "any talks for data engineer?"

![](https://i.postimg.cc/VLWx9S5z/sample-nlu.jpg)

Alternatively, if you have a test set, you can use it to evaluate the performance of the NLU component by executing the following command in a shell:

    rasa test nlu -u path/to/test_data

## Train Rasa's dialogue management component (Core)

The goal of this section to train the bot on how to respond at every turn of a conversation with a user.

As usual, we start by providing some training data. In Rasa, the data used to train the dialogue manager is called "stories". Click [here](https://rasa.com/docs/rasa/core/stories/) for details on how to format stories.

In [10]:
stories_md = """
## happy path all info 1               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* thanks
  - utter_thanks
  
  
## happy path all info 2               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks    
  

## happy path all info 3               
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker
  - utter_speaker
* length
  - utter_length
* thanks
  - utter_thanks  


## happy path no relevant audience            
* greet              
  - utter_greet
* recommend_session
  - utter_ask_relevant_audience
* inform{"relevant_audience":"Data Scientists"}
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* length
  - utter_length
* speaker
  - utter_speaker
* thanks
  - utter_thanks

"""

%store stories_md > ./data/stories.md

Writing 'stories_md' (str) to file './data/stories.md'.


We also need to update `config.yml` to include policies for dialogue management. Click [here](https://rasa.com/docs/rasa/core/policies/) to learn more about the available built-in policies. 

In [11]:
policies_md = """

policies:
  - name: MemoizationPolicy
  - name: KerasPolicy
    epochs: 300
  - name: MappingPolicy

"""

%store policies_md >> ./config.yml

Writing 'policies_md' (str) to file './config.yml'.


Finally, we need to create a `domain.yml` file to define all the things the bot can do. Click [here](https://rasa.com/docs/rasa/core/domains/) for more details.

In [12]:
domain_yml = """

intents:
- greet
- goodbye
- recommend_session
- speaker
- length
- abstract
- thanks
- inform


slots:
  relevant_audience:
    type: text
  length:
    type: unfeaturized
  speaker:
    type: unfeaturized
  abstract:
    type: unfeaturized


entities:
- relevant_audience


actions:
- utter_greet
- utter_thanks
- utter_goodbye
- utter_length
- utter_abstract
- utter_speaker
- utter_ask_relevant_audience
- utter_ask_other_questions
- action_recommend_session


templates:
  utter_greet:
    - text: "Hey! I am a conference assistant. I can help you find the sessions to attend, or answer conference-related questions."

  utter_thanks:
    - text: "You are very welcome!"

  utter_goodbye:
    - text: "See you later"
    - text: "Bye bye"
  
  utter_ask_other_questions:
    - text: "Would you like to know anything else?"
    
  utter_ask_relevant_audience:
    - text: "What would be the relevant audience?"
  
  utter_speaker:
    - text: "The speaker is {speaker}."
  
  utter_length:
    - text: "The session is {length}"
    
  utter_abstract:
    - text: "{abstract}"

"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


Now we can train the dialogue management model:

In [13]:
!rasa train # we don't call "rasa train core" because we want the output file to contain both the nlu and core model

2019-07-22 12:40:16 INFO     rasa.model  - Data (core-config) for Core model changed.
Training Core model...
Processed trackers: 100%|██████████| 4/4 [00:00<00:00, 688.95it/s, # actions=27]
Processed actions: 27it [00:00, 3036.99it/s, # examples=27]
Processed trackers: 100%|████████| 66/66 [00:00<00:00, 162.71it/s, # actions=43]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 27)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7680      
_________________________________________________________________
dense (Dense)                (None, 17)                561       
_________________________________________________________________
activation (Activation)      (None, 17)                0         
Total params: 8,241
Trainable params: 8,241
Non-trainable params: 0
_______

43/43 [==============================] - 0s 906us/sample - loss: 1.4912 - acc: 0.5116
Epoch 68/300
43/43 [==============================] - 0s 825us/sample - loss: 1.4774 - acc: 0.5116
Epoch 69/300
43/43 [==============================] - 0s 754us/sample - loss: 1.5128 - acc: 0.5116
Epoch 70/300
43/43 [==============================] - 0s 799us/sample - loss: 1.4774 - acc: 0.5116
Epoch 71/300
43/43 [==============================] - 0s 1ms/sample - loss: 1.4819 - acc: 0.5116
Epoch 72/300
43/43 [==============================] - 0s 870us/sample - loss: 1.4512 - acc: 0.5116
Epoch 73/300
43/43 [==============================] - 0s 615us/sample - loss: 1.4304 - acc: 0.5116
Epoch 74/300
43/43 [==============================] - 0s 916us/sample - loss: 1.4272 - acc: 0.5116
Epoch 75/300
43/43 [==============================] - 0s 937us/sample - loss: 1.4711 - acc: 0.5116
Epoch 76/300
43/43 [==============================] - 0s 706us/sample - loss: 1.4105 - acc: 0.5116
Epoch 77/300
43/43 [=====

43/43 [==============================] - 0s 915us/sample - loss: 0.9397 - acc: 0.6279
Epoch 151/300
43/43 [==============================] - 0s 837us/sample - loss: 0.9284 - acc: 0.6047
Epoch 152/300
43/43 [==============================] - 0s 958us/sample - loss: 0.8801 - acc: 0.6744
Epoch 153/300
43/43 [==============================] - 0s 721us/sample - loss: 0.9625 - acc: 0.6512
Epoch 154/300
43/43 [==============================] - 0s 713us/sample - loss: 0.9143 - acc: 0.6744
Epoch 155/300
43/43 [==============================] - 0s 874us/sample - loss: 0.9179 - acc: 0.6512
Epoch 156/300
43/43 [==============================] - 0s 1ms/sample - loss: 0.8933 - acc: 0.6744
Epoch 157/300
43/43 [==============================] - 0s 1ms/sample - loss: 0.8884 - acc: 0.6047
Epoch 158/300
43/43 [==============================] - 0s 2ms/sample - loss: 0.9123 - acc: 0.6512
Epoch 159/300
43/43 [==============================] - 0s 766us/sample - loss: 0.8912 - acc: 0.6744
Epoch 160/300
43/43 

Epoch 233/300
43/43 [==============================] - 0s 647us/sample - loss: 0.5120 - acc: 0.9070
Epoch 234/300
43/43 [==============================] - 0s 492us/sample - loss: 0.4926 - acc: 0.8837
Epoch 235/300
43/43 [==============================] - 0s 693us/sample - loss: 0.4469 - acc: 0.9535
Epoch 236/300
43/43 [==============================] - 0s 775us/sample - loss: 0.4409 - acc: 0.9070
Epoch 237/300
43/43 [==============================] - 0s 613us/sample - loss: 0.4548 - acc: 0.8837
Epoch 238/300
43/43 [==============================] - 0s 758us/sample - loss: 0.4904 - acc: 0.8372
Epoch 239/300
43/43 [==============================] - 0s 689us/sample - loss: 0.4640 - acc: 0.8837
Epoch 240/300
43/43 [==============================] - 0s 455us/sample - loss: 0.4424 - acc: 0.9302
Epoch 241/300
43/43 [==============================] - 0s 418us/sample - loss: 0.3943 - acc: 0.9070
Epoch 242/300
43/43 [==============================] - 0s 601us/sample - loss: 0.4160 - acc: 0.9070


Before we can talk to our bot, we need to implement any custom action. In example, the only custom action we need to implement is `action_recommend_session`.

Sessions will be recommended based on data stored in an SQLite database. Here is what the data looks like:

In [14]:
conn = lite.connect('../database/ConfDB.db')
data = pd.read_sql('SELECT * from agenda', conn)
conn.close()

data.sample(n=10)

,session_name,session_type,speaker_name,company,abstract,venue,begin_date,relevant_audience,length
89,Using AutoML to automate selection of machine ...,40-minute session,Francesca Lazzeri and Wee Hyong Tok,Microsoft and Microsoft,Automated machine learning (AutoML) enables bo...,Grand Ballroom West,2019-04-18 13:00:00,"Data scientists, data engineers, and AI develo...",
61,AI at Night,Event,,,"Don't miss AI at Night, happening on Wednesday...",Bryant Park Grill,2019-04-17 19:00:00,,
116,Make music composing easier for amateurs: A hy...,40-minute session,Andrew Caosun,Horace Mann School,We propose a framework that unifies Hidden Mar...,Regent Parlor,2019-04-18 16:55:00,Entrepreneurs and those interested in computer...,
72,Regularization of RNN through Bayesian networks,40-minute session,vishal hawa,Vanguard,While Deep Learning has shown significant prom...,Sutton South,2019-04-18 11:05:00,"Data scientists, marketing managers, and busin...",
70,Keynote by Olga Troyanskaya,Keynote,Olga Troyanskaya,Princeton University,Keynote by Olga Troyanskaya,Grand Ballroom West,2019-04-18 10:15:00,,
86,Risk-free deep learning without sacrificing pe...,40-minute session,Evan Sparks,Determined AI,Building deep learning applications is hard. B...,Mercury Rotunda,2019-04-18 13:00:00,"Data scientists, machine learning engineers, a...",
63,Opening Remarks,Keynote,Ben Lorica and Roger Chen,O'Reilly Media and Computable,Opening Remarks,Grand Ballroom West,2019-04-18 9:00:00,,
103,Distributed TensorFlow with distribution strat...,40-minute session,Magnus Hyttsten,Google,Session description: This session covers how t...,Mercury Rotunda,2019-04-18 14:40:00,Developers and software engineers,
17,TensorFlow 2.0: Machine learning for you,40-minute session,joshua gordon,Google,Learn about the very latest in TensorFlow dire...,Mercury Rotunda,2019-04-17 11:05:00,"Developers, software engineers, decision makers",40 min
73,Executive Briefing: The hidden data in AI IP,40-minute session,Thomas G Marlow,Black Hills IP,Three elements will control the AI market - Te...,Mercury Ballroom,2019-04-18 11:05:00,Managers and executives creating products and ...,


Let's copy the database to the bot's root directory:

In [15]:
%cp ../database/ConfDB.db ./ConfDB.db

To implement a custom action, we just need to subclass the `Action` class and implement the `name` and `run` method in a file named `actions.py`:

In [16]:
actions = """
import random
import sqlite3 as lite

from rasa_sdk import Action
from rasa_sdk.events import SlotSet


class ActionRecommendTalk(Action):
    def name(self):
        return "action_recommend_session"

    def run(self, dispatcher, tracker, domain):
        conn = lite.connect('./ConfDB.db')
        cur = conn.cursor()

        relevant_audience = tracker.get_slot("relevant_audience")

        cur.execute(f"SELECT * from agenda WHERE relevant_audience like '%{relevant_audience}%'")

        results = cur.fetchall()

        ind = random.randint(0, len(results))
        recommend_talk = list(results[ind])

        title, _, speaker, _, abstract, *_, length = recommend_talk

        dispatcher.utter_message(f"I would recommend you to attend {title}")

        return [SlotSet("speaker", speaker),
                SlotSet("abstract", abstract),
                SlotSet("length", length)]
"""

%store actions > actions.py

Writing 'actions' (str) to file 'actions.py'.


Remember to uncomment the `action_endpoint` section in the `endpoints.yml` file:

In [17]:
# before
!cat endpoints.yml

# This file contains the different endpoints your bot can use.

# Server where the models are pulled from.
# https://rasa.com/docs/rasa/user-guide/running-the-server/#fetching-models-from-a-server/

#models:
#  url: http://my-server.com/models/default_core@latest
#  wait_time_between_pulls:  10   # [optional](default: 100)

# Server which runs your custom actions.
# https://rasa.com/docs/rasa/core/actions/#custom-actions/

#action_endpoint:
#  url: "http://localhost:5055/webhook"

# Tracker store which is used to store the conversations.
# By default the conversations are stored in memory.
# https://rasa.com/docs/rasa/api/tracker-stores/

#tracker_store:
#    type: redis
#    url: <host of the redis instance, e.g. localhost>
#    port: <port of your redis instance, usually 6379>
#    db: <number of your database within redis, e.g. 0>
#    password: <password used for authentication>

#tracker_store:
#    type: mongod
#    url: <url to your mongo instance, e.g. mongodb://localhost:27017

In [18]:
# use sed to remove the comments
!sed -i '/action_endpoint:/s/^#//' endpoints.yml
!sed -i '/ url: ".*"/s/^#//' endpoints.yml

In [19]:
# after
!cat endpoints.yml

# This file contains the different endpoints your bot can use.

# Server where the models are pulled from.
# https://rasa.com/docs/rasa/user-guide/running-the-server/#fetching-models-from-a-server/

#models:
#  url: http://my-server.com/models/default_core@latest
#  wait_time_between_pulls:  10   # [optional](default: 100)

# Server which runs your custom actions.
# https://rasa.com/docs/rasa/core/actions/#custom-actions/

action_endpoint:
  url: "http://localhost:5055/webhook"

# Tracker store which is used to store the conversations.
# By default the conversations are stored in memory.
# https://rasa.com/docs/rasa/api/tracker-stores/

#tracker_store:
#    type: redis
#    url: <host of the redis instance, e.g. localhost>
#    port: <port of your redis instance, usually 6379>
#    db: <number of your database within redis, e.g. 0>
#    password: <password used for authentication>

#tracker_store:
#    type: mongod
#    url: <url to your mongo instance, e.g. mongodb://localhost:27017>


Now, we are ready to interact with the bot.

In a terminal, execute:

    rasa run actions

to start the action server and then execute:

    rasa shell --endpoints endpoints.yml

to talk to the bot via the command line.

# Advanced

## Handling synonym

Sometimes, people write "machine learning engineer" as "ML engineer" or "software engineer" as SWE. To account for synonyms, we need to modify the training data for the NLU component in the following way:

In [20]:
nlu_md = """
## synonym:machine learning engineer
- ML Engineer

## synonym:software engineer
- SWE

"""

%store nlu_md >> ./data/nlu.md

Writing 'nlu_md' (str) to file './data/nlu.md'.


We also need to update our NLU pipeline so that it calls the synonym mapper component:

In [21]:
configuration = """

language: "en"

pipeline:
- name: "WhitespaceTokenizer"
- name: "CRFEntityExtractor"
- name: "CountVectorsFeaturizer"
- name: "EmbeddingIntentClassifier"
  epochs: 300
- name: "EntitySynonymMapper"


policies:
  - name: MemoizationPolicy
  - name: KerasPolicy
    epochs: 300
  - name: MappingPolicy

""" 

%store configuration > config.yml

Writing 'configuration' (str) to file 'config.yml'.


In [22]:
!rasa train

2019-07-22 12:40:48 INFO     rasa.model  - Data (nlu-config) for NLU model changed.
Core stories/configuration did not change. No need to retrain Core model.
Training NLU model...
2019-07-22 12:40:49 INFO     rasa.nlu.training_data.loading  - Training data format of '/tmp/tmp9ztv_po2/609461977388457eb9e47f996241aaad_nlu.md' is 'md'.
2019-07-22 12:40:49 INFO     rasa.nlu.training_data.training_data  - Training data stats: 
	- intent examples: 37 (8 distinct intents)
	- Found intents: 'thanks', 'inform', 'length', 'greet', 'recommend_session', 'goodbye', 'abstract', 'speaker'
	- entity examples: 7 (1 distinct entities)
	- found entities: 'relevant_audience'

2019-07-22 12:40:49 INFO     rasa.nlu.model  - Starting to train component WhitespaceTokenizer
2019-07-22 12:40:49 INFO     rasa.nlu.model  - Finished training component.
2019-07-22 12:40:49 INFO     rasa.nlu.model  - Starting to train component CRFEntityExtractor
2019-07-22 12:40:49 INFO     rasa.nlu.model  - Finished training compo

Testing it with `rasa shell nlu` shows that it kinda works...

![](https://i.postimg.cc/4nkMg9PF/sample-synonym-1.jpg)

![](https://i.postimg.cc/W12VFBmY/sample-synonym-2.jpg)

![](https://i.postimg.cc/NFqqgSf4/sample-synonym-3.jpg)

## Handling multiple-intents

To support user input containing multiple intent, we need to add additonal NLU training data where the intent is the concatenation of 2 (or more) intents:

In [26]:
nlu_md = """

## intent: speaker+length
- Who is the speaker and how long is the talk?
- Tell me the speaker name. Also, how long is the talk?
- Tell me the name of the presenter and how long is this session?

"""

%store nlu_md >> ./data/nlu.md

Writing 'nlu_md' (str) to file './data/nlu.md'.


Next, we need to update the NLU pipeline to tell the intent classifier to expect multi-intent inputs:

In [24]:
configuration = """

language: "en"

pipeline:
- name: "WhitespaceTokenizer"
- name: "CRFEntityExtractor"
- name: "CountVectorsFeaturizer"
- name: "EmbeddingIntentClassifier"
  epochs: 300
  intent_tokenization_flag: true
  intent_split_symbol: "+"
- name: "EntitySynonymMapper"


policies:
  - name: MemoizationPolicy
  - name: KerasPolicy
    epochs: 300
  - name: MappingPolicy

""" 

%store configuration > config.yml

Writing 'configuration' (str) to file 'config.yml'.


We also need to update `domain.yml` to include the new intent:

In [28]:
domain_yml = """

intents:
- greet
- goodbye
- recommend_session
- speaker
- length
- abstract
- thanks
- inform
- speaker+length


slots:
  relevant_audience:
    type: text
  length:
    type: unfeaturized
  speaker:
    type: unfeaturized
  abstract:
    type: unfeaturized


entities:
- relevant_audience


actions:
- utter_greet
- utter_thanks
- utter_goodbye
- utter_length
- utter_abstract
- utter_speaker
- utter_ask_relevant_audience
- utter_ask_other_questions
- action_recommend_session


templates:
  utter_greet:
    - text: "Hey! I am a conference assistant. I can help you find the sessions to attend, or answer conference-related questions."

  utter_thanks:
    - text: "You are very welcome!"

  utter_goodbye:
    - text: "See you later"
    - text: "Bye bye"
  
  utter_ask_other_questions:
    - text: "Would you like to know anything else?"
    
  utter_ask_relevant_audience:
    - text: "What would be the relevant audience?"
  
  utter_speaker:
    - text: "The speaker is {speaker}."
  
  utter_length:
    - text: "The session is {length}"
    
  utter_abstract:
    - text: "{abstract}"

"""

%store domain_yml > domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


Finally, we update the `stories.md` with some examples where a user gives a multi-intent input:

In [30]:
stories_md = """

## multi-intent story1              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* thanks
  - utter_thanks
  
## multi-intent story2              
* greet              
  - utter_greet
* recommend_session{"relevant_audience":"Data Scientists"}               
  - action_recommend_session
  - slot{"speaker":"Justina"}
  - slot{"length":"5 min"}
  - slot{"abstract":"Workshop on chatbots"}
* speaker+length
  - utter_length
  - utter_speaker
* abstract
  - utter_abstract
* thanks
  - utter_thanks
  
"""

%store stories_md >> ./data/stories.md

Writing 'stories_md' (str) to file './data/stories.md'.


Now we retrain everything to incorporate the latest changes:

In [32]:
!rasa train

2019-07-22 14:14:12 INFO     rasa.model  - Data (stories) for Core model changed.
2019-07-22 14:14:12 INFO     rasa.model  - Data (messages) for NLU model changed.
Training Core model...
Processed trackers: 100%|██████████| 7/7 [00:00<00:00, 182.59it/s, # actions=38]
Processed actions: 38it [00:00, 992.67it/s, # examples=38]
Processed trackers: 100%|███████| 188/188 [00:02<00:00, 91.33it/s, # actions=66]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 28)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7808      
_________________________________________________________________
dense (Dense)                (None, 17)                561       
_________________________________________________________________
activation (Activation)      (None, 17)                0        

66/66 [==============================] - 0s 2ms/sample - loss: 1.5411 - acc: 0.4848
Epoch 68/300
66/66 [==============================] - 0s 5ms/sample - loss: 1.5320 - acc: 0.4848
Epoch 69/300
66/66 [==============================] - 0s 3ms/sample - loss: 1.5235 - acc: 0.4848
Epoch 70/300
66/66 [==============================] - 0s 2ms/sample - loss: 1.5178 - acc: 0.4848
Epoch 71/300
66/66 [==============================] - 0s 2ms/sample - loss: 1.5090 - acc: 0.4848
Epoch 72/300
66/66 [==============================] - 0s 2ms/sample - loss: 1.5075 - acc: 0.4848
Epoch 73/300
66/66 [==============================] - 0s 6ms/sample - loss: 1.5014 - acc: 0.4848
Epoch 74/300
66/66 [==============================] - 0s 5ms/sample - loss: 1.4999 - acc: 0.4848
Epoch 75/300
66/66 [==============================] - 0s 2ms/sample - loss: 1.4819 - acc: 0.4848
Epoch 76/300
66/66 [==============================] - 0s 2ms/sample - loss: 1.4744 - acc: 0.4848
Epoch 77/300
66/66 [=======================

66/66 [==============================] - 0s 7ms/sample - loss: 0.8486 - acc: 0.7273
Epoch 152/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.8165 - acc: 0.7273
Epoch 153/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.7723 - acc: 0.7576
Epoch 154/300
66/66 [==============================] - 0s 1ms/sample - loss: 0.8244 - acc: 0.7273
Epoch 155/300
66/66 [==============================] - 1s 9ms/sample - loss: 0.7432 - acc: 0.7879
Epoch 156/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.7486 - acc: 0.7576
Epoch 157/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.8119 - acc: 0.6818
Epoch 158/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.8095 - acc: 0.7273
Epoch 159/300
66/66 [==============================] - 0s 4ms/sample - loss: 0.7522 - acc: 0.7273
Epoch 160/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.7800 - acc: 0.7576
Epoch 161/300
66/66 [=============

66/66 [==============================] - 0s 6ms/sample - loss: 0.3638 - acc: 0.9091
Epoch 235/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.3586 - acc: 0.9242
Epoch 236/300
66/66 [==============================] - 0s 1ms/sample - loss: 0.3534 - acc: 0.9242
Epoch 237/300
66/66 [==============================] - 0s 1ms/sample - loss: 0.3484 - acc: 0.9394
Epoch 238/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.3628 - acc: 0.9091
Epoch 239/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.3513 - acc: 0.9394
Epoch 240/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.3466 - acc: 0.9091
Epoch 241/300
66/66 [==============================] - 0s 3ms/sample - loss: 0.3270 - acc: 0.9242
Epoch 242/300
66/66 [==============================] - 0s 3ms/sample - loss: 0.3391 - acc: 0.8788
Epoch 243/300
66/66 [==============================] - 0s 2ms/sample - loss: 0.3155 - acc: 0.9545
Epoch 244/300
66/66 [=============

Epochs: 100%|██████████| 300/300 [00:09<00:00, 32.33it/s, loss=0.092, acc=1.000]
2019-07-22 14:15:49 INFO     rasa.nlu.classifiers.embedding_intent_classifier  - Finished training embedding classifier, loss=0.092, train accuracy=1.000
2019-07-22 14:15:49 INFO     rasa.nlu.model  - Finished training component.
2019-07-22 14:15:49 INFO     rasa.nlu.model  - Starting to train component EntitySynonymMapper
2019-07-22 14:15:49 INFO     rasa.nlu.model  - Finished training component.
2019-07-22 14:15:50 INFO     rasa.nlu.model  - Successfully saved model into '/tmp/tmpnyhm0xpf/nlu'
NLU model training completed.
Your Rasa model is trained and saved at '/app/demo_confbot/models/20190722-141550.tar.gz'.


## Spellcheck

## Fallback policy